# Using custom containers with AI Platform Training

In this lab, you develop, package as a docker image, and run on **AI Platform Training** a training application that trains a multi-class classification model that predicts the type of forest cover from cartographic data. The [dataset](../datasets/covertype/README.md) used in the lab is based on **Covertype Data Set** from UCI Machine Learning Repository.

The training code uses `scikit-learn` for data pre-processing and modeling. The code has been instrumented using the `hypertune` package so it can be used with **AI Platform** hyperparameter tuning.


In [1]:
import json
import os
import numpy as np
import pandas as pd
import pickle
import uuid
import time
import tempfile

from googleapiclient import discovery
from googleapiclient import errors

from google.cloud import bigquery
from jinja2 import Template
from kfp.components import func_to_container_op
from typing import NamedTuple

from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

## Configure environment settings

Set location paths, connections strings, and other environment settings. Make sure to update `PROJECT_ID`, `NAME_PREFIX` and `REGION` with your settings. Recall that `NAME_PREFIX` is the parameter used during the lab environment setup that was used in constructing names of the environment's resources.

In [2]:
PROJECT_ID = 'mlops-workshop'
NAME_PREFIX = 'mlops-env20'
REGION = 'us-central1'

DATA_ROOT = 'gs://workshop-datasets/covertype'
TRAINING_FILE_PATH = DATA_ROOT + '/training/dataset.csv'
VALIDATION_FILE_PATH = DATA_ROOT + '/evaluation/dataset.csv'

# Job dir for AI Platform Training
JOB_DIR_ROOT='gs://{}-artifact-store/jobs'.format(NAME_PREFIX)

## Explore the Covertype dataset 

In [3]:
df_train = pd.read_csv(TRAINING_FILE_PATH)
df_validation = pd.read_csv(VALIDATION_FILE_PATH)
df_train

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2989,329,11,342,74,2861,194,227,172,1471,Commanche,C7756,2
1,3223,264,7,228,41,968,203,243,179,190,Commanche,C7756,1
2,3208,322,22,162,13,2876,158,212,191,2486,Rawah,C7745,1
3,3041,52,11,67,4,2467,226,217,124,3036,Commanche,C7700,2
4,2988,259,28,242,75,5197,143,242,231,1425,Rawah,C7745,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
431004,2476,67,31,268,122,666,233,163,39,721,Commanche,C4703,2
431005,3029,130,7,30,4,3266,233,236,136,5388,Rawah,C7102,2
431006,3017,107,9,242,46,4265,235,230,126,1052,Rawah,C7746,2
431007,3314,144,16,362,56,1549,241,235,119,2911,Rawah,C7745,1


In [4]:
print(df_train.shape)
print(df_validation.shape)

(431009, 13)
(75000, 13)


## Develop the training application

### Configure the `sklearn` training pipeline.

The training pipeline preprocesses data by standardizing all numeric features using `sklearn.preprocessing.StandardScaler` and encoding all categorical features using `sklearn.preprocessing.OneHotEncoder`. It uses stochastic gradient descent linear classifier (`SGDClassifier`) for modeling.

In [5]:
numeric_feature_indexes = slice(0, 10)
categorical_feature_indexes = slice(10, 12)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_feature_indexes),
        ('cat', OneHotEncoder(), categorical_feature_indexes) 
    ])

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SGDClassifier(loss='log'))
])

### Convert all numeric features to `float64`

To avoid warning messages from `StandardScaler` all numeric features are converted to `float64`.

In [6]:
num_features_type_map = {feature: 'float64' for feature in df_train.columns[numeric_feature_indexes]}

df_train = df_train.astype(num_features_type_map)
df_validation = df_validation.astype(num_features_type_map)

### Run the pipeline locally.

In [8]:
X_train = df_train.drop('Cover_Type', axis=1)
y_train = df_train['Cover_Type']
X_validation = df_validation.drop('Cover_Type', axis=1)
y_validation = df_validation['Cover_Type']

pipeline.set_params(classifier__alpha=0.001, classifier__max_iter=200)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('preprocessor', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('num', StandardScaler(copy=True, with_mean=True, with_std=True), slice(0, 10, None)), ('cat', OneHotEncoder(categorical_features=None, categories=...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))])

### Calculate the trained model's accuracy.

In [9]:
accuracy = pipeline.score(X_validation, y_validation)
print(accuracy)

0.7044266666666666


### Prepare the hyperparameter tuning application.
Since the training run on this dataset is computationally expensive you can benefit from running a distributed hyperparameter tuning job on AI Platform Training.

In [11]:
TRAINING_APP_FOLDER = 'training_app'
os.makedirs(TRAINING_APP_FOLDER, exist_ok=True)

### Write the tuning script. 

Notice the use of the `hypertune` package to report the `accuracy` optimization metric to AI Platform hyperparameter tuning service.

In [12]:
%%writefile {TRAINING_APP_FOLDER}/train.py

import os
import subprocess
import sys

import fire
import pickle
import numpy as np
import pandas as pd

import hypertune

from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder


def train_evaluate(job_dir, training_dataset_path, validation_dataset_path, alpha, max_iter, hptune):
    
  df_train = pd.read_csv(training_dataset_path)
  df_validation = pd.read_csv(validation_dataset_path)
    
  if not hptune:
    df_train = pd.concat([df_train, df_validation])

  numeric_feature_indexes = slice(0, 10)
  categorical_feature_indexes = slice(10, 12)

  preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_feature_indexes),
        ('cat', OneHotEncoder(), categorical_feature_indexes) 
    ])

  pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SGDClassifier(loss='log'))
  ])
    
  num_features_type_map = {feature: 'float64' for feature in df_train.columns[numeric_feature_indexes]}
  df_train = df_train.astype(num_features_type_map)
  df_validation = df_validation.astype(num_features_type_map) 

  print('Starting training: alpha={}, max_iter={}'.format(alpha, max_iter))
  X_train = df_train.drop('Cover_Type', axis=1)
  y_train = df_train['Cover_Type']
  
  pipeline.set_params(classifier__alpha=alpha, classifier__max_iter=max_iter)
  pipeline.fit(X_train, y_train)
  
  if hptune:
    X_validation = df_validation.drop('Cover_Type', axis=1)
    y_validation = df_validation['Cover_Type']
    accuracy = pipeline.score(X_validation, y_validation)
    print('Model accuracy: {}'.format(accuracy))
    # Log it with hypertune
    hpt = hypertune.HyperTune()
    hpt.report_hyperparameter_tuning_metric(
      hyperparameter_metric_tag='accuracy',
      metric_value=accuracy
    )

  # Save the model
  if not hptune:
    model_filename = 'model.pkl'
    with open(model_filename, 'wb') as model_file:
        pickle.dump(pipeline, model_file)
    gcs_model_path = "{}/{}".format(job_dir, model_filename)
    subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path], stderr=sys.stdout)
    print("Saved model in: {}".format(gcs_model_path)) 
    
if __name__ == "__main__":
  fire.Fire(train_evaluate)

Overwriting training_app/train.py


### Package the script into a docker image.

Notice that the training image is a derivative of `mlops-dev:TF115-TFX015-KFP136`. The reason is to make sure that the development environment (your AI Platform Notebook instance) and the AI Platform Training environment are consistent. Since the AI Platform Notebook instance is based on the `mlops-dev:TF115-TFX015-KFP136` image we use the same image as a base for the training image. 

Make sure to update the URI for the base image so that it points to your project's **Container Registry**.

In [13]:
%%writefile {TRAINING_APP_FOLDER}/Dockerfile

FROM gcr.io/mlops-workshop/mlops-dev:TF115-TFX015-KFP136
RUN pip install -U fire cloudml-hypertune
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]

Overwriting training_app/Dockerfile


### Build the docker image. 

You use **Cloud Build** to build the image and push it your project's **Container Registry**. As you use the remote cloud service to build the image, you don't need a local installation of Docker.

In [14]:
IMAGE_NAME='trainer_image'
IMAGE_TAG='latest'
IMAGE_URI='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, IMAGE_TAG)

!gcloud builds submit --tag $IMAGE_URI $TRAINING_APP_FOLDER

Creating temporary tarball archive of 3 file(s) totalling 2.9 KiB before compression.
Uploading tarball of [training_app] to [gs://mlops-workshop_cloudbuild/source/1580783167.88-30105d8eb2cc41fd8159803acca592e4.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/mlops-workshop/builds/158e81d4-da6f-40b0-8eb7-5913fd7c117a].
Logs are available at [https://console.cloud.google.com/gcr/builds/158e81d4-da6f-40b0-8eb7-5913fd7c117a?project=745302968357].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "158e81d4-da6f-40b0-8eb7-5913fd7c117a"

FETCHSOURCE
Fetching storage object: gs://mlops-workshop_cloudbuild/source/1580783167.88-30105d8eb2cc41fd8159803acca592e4.tgz#1580783168283926
Copying gs://mlops-workshop_cloudbuild/source/1580783167.88-30105d8eb2cc41fd8159803acca592e4.tgz#1580783168283926...
/ [1 files][  1.4 KiB/  1.4 KiB]                                                
Operation completed over 1 objects/1.4 KiB.                     

## Submit the AI Platform hyperparameter tuning job

### Create the hyperparameter configuration file. 
Recall that the training code uses `SGDClassifier`. The training application has been designed to accept two hyperparameters that control `SGDClassifier`:
- Max iterations
- Alpha

The below file configures AI Platform hypertuning to run up to 6 trials on up to three nodes and to choose from two discrete values of `max_iter` and the linear range betwee 0.00001 and 0.001 for `alpha`.

In [15]:
%%writefile {TRAINING_APP_FOLDER}/hptuning_config.yaml

trainingInput:
  hyperparameters:
    goal: MAXIMIZE
    maxTrials: 6
    maxParallelTrials: 3
    hyperparameterMetricTag: accuracy
    enableTrialEarlyStopping: TRUE 
    params:
    - parameterName: max_iter
      type: DISCRETE
      discreteValues: [
          200,
          500
          ]
    - parameterName: alpha
      type: DOUBLE
      minValue:  0.00001
      maxValue:  0.001
      scaleType: UNIT_LINEAR_SCALE

Overwriting training_app/hptuning_config.yaml


### Start the hyperparameter tuning job.

Use the `gcloud` command to start the hyperparameter tuning job.

In [16]:
JOB_NAME = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
JOB_DIR = "{}/{}".format(JOB_DIR_ROOT, JOB_NAME)
SCALE_TIER = "BASIC"

!gcloud ai-platform jobs submit training $JOB_NAME \
--region=$REGION \
--job-dir=$JOB_DIR \
--master-image-uri=$IMAGE_URI \
--scale-tier=$SCALE_TIER \
--config $TRAINING_APP_FOLDER/hptuning_config.yaml \
-- \
--training_dataset_path=$TRAINING_FILE_PATH \
--validation_dataset_path=$VALIDATION_FILE_PATH \
--hptune

Job [JOB_20200204_022944] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe JOB_20200204_022944

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs JOB_20200204_022944
jobId: JOB_20200204_022944
state: QUEUED


### Monitor the job.

You can monitor the job using GCP console or from within the notebook using `gcloud` commands.

In [17]:
!gcloud ai-platform jobs describe $JOB_NAME

createTime: '2020-02-04T02:29:46Z'
etag: nMW7Qju5haI=
jobId: JOB_20200204_022944
startTime: '2020-02-04T02:29:50Z'
state: RUNNING
trainingInput:
  args:
  - --training_dataset_path=gs://workshop-datasets/covertype/training/dataset.csv
  - --validation_dataset_path=gs://workshop-datasets/covertype/evaluation/dataset.csv
  - --hptune
  hyperparameters:
    enableTrialEarlyStopping: true
    goal: MAXIMIZE
    hyperparameterMetricTag: accuracy
    maxParallelTrials: 3
    maxTrials: 6
    params:
    - discreteValues:
      - 200.0
      - 500.0
      parameterName: max_iter
      type: DISCRETE
    - maxValue: 0.001
      minValue: 1e-05
      parameterName: alpha
      scaleType: UNIT_LINEAR_SCALE
      type: DOUBLE
  jobDir: gs://mlops-env20-artifact-store/jobs/JOB_20200204_022944
  masterConfig:
    imageUri: gcr.io/mlops-workshop/trainer_image:latest
  region: us-central1
trainingOutput:
  hyperparameterMetricTag: accuracy
  isHyperparameterTuningJob: true

View job in the Cloud Cons

In [18]:
!gcloud ai-platform jobs stream-logs $JOB_NAME

INFO	2020-02-04 02:29:45 +0000	service		Validating job requirements...
INFO	2020-02-04 02:29:46 +0000	service		Job creation request has been successfully validated.
INFO	2020-02-04 02:29:46 +0000	service		Job JOB_20200204_022944 is queued.
INFO	2020-02-04 02:29:56 +0000	service	1	Waiting for job to be provisioned.
INFO	2020-02-04 02:29:56 +0000	service	2	Waiting for job to be provisioned.
INFO	2020-02-04 02:29:56 +0000	service	3	Waiting for job to be provisioned.
INFO	2020-02-04 02:29:59 +0000	service	3	Waiting for training program to start.
INFO	2020-02-04 02:29:59 +0000	service	1	Waiting for training program to start.
INFO	2020-02-04 02:29:59 +0000	service	2	Waiting for training program to start.
INFO	2020-02-04 02:37:44 +0000	master-replica-0	2	Starting training: alpha=0.00012374698042869569, max_iter=200
INFO	2020-02-04 02:37:44 +0000	master-replica-0	2	Model accuracy: 0.71268
INFO	2020-02-04 02:41:02 +0000	service	2	Job completed successfully.
INFO	2020-02-04 02:42:46 +0000	servic

### Retrieve HP-tuning results.

After the job completes you can review the results using GCP Console or programatically by calling the AI Platform Training REST end-point.

In [19]:
ml = discovery.build('ml', 'v1')

job_id = 'projects/{}/jobs/{}'.format(PROJECT_ID, JOB_NAME)
request = ml.projects().jobs().get(name=job_id)

try:
    response = request.execute()
except errors.HttpError as err:
    print(err)
except:
    print("Unexpected error")
    
response

{'createTime': '2020-02-04T02:29:46Z',
 'endTime': '2020-02-04T03:02:56Z',
 'etag': 'gULvrwwaArQ=',
 'jobId': 'JOB_20200204_022944',
 'startTime': '2020-02-04T02:29:50Z',
 'state': 'SUCCEEDED',
 'trainingInput': {'args': ['--training_dataset_path=gs://workshop-datasets/covertype/training/dataset.csv',
   '--validation_dataset_path=gs://workshop-datasets/covertype/evaluation/dataset.csv',
   '--hptune'],
  'hyperparameters': {'enableTrialEarlyStopping': True,
   'goal': 'MAXIMIZE',
   'hyperparameterMetricTag': 'accuracy',
   'maxParallelTrials': 3,
   'maxTrials': 6,
   'params': [{'discreteValues': [200, 500],
     'parameterName': 'max_iter',
     'type': 'DISCRETE'},
    {'maxValue': 0.001,
     'minValue': 1e-05,
     'parameterName': 'alpha',
     'scaleType': 'UNIT_LINEAR_SCALE',
     'type': 'DOUBLE'}]},
  'jobDir': 'gs://mlops-env20-artifact-store/jobs/JOB_20200204_022944',
  'masterConfig': {'imageUri': 'gcr.io/mlops-workshop/trainer_image:latest'},
  'region': 'us-central1'},

The returned run results are sorted by a value of the optimization metric. The best run is the first item on the returned list.

In [21]:
response['trainingOutput']['trials'][0]

{'endTime': '2020-02-04T03:00:33Z',
 'finalMetric': {'objectiveValue': 0.7132933333333333, 'trainingStep': '1'},
 'hyperparameters': {'alpha': '9.0244140624999992e-05', 'max_iter': '200'},
 'startTime': '2020-02-04T02:49:10.486348046Z',
 'state': 'SUCCEEDED',
 'trialId': '6'}

## Retrain the model with the best hyperparameters

You can now retrain the model using the best hyperparameters and using combined training and validation splits as a training dataset.

### Configure and run the training job

In [22]:
alpha = response['trainingOutput']['trials'][0]['hyperparameters']['alpha']
max_iter = response['trainingOutput']['trials'][0]['hyperparameters']['max_iter']

In [23]:
JOB_NAME = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
JOB_DIR = "{}/{}".format(JOB_DIR_ROOT, JOB_NAME)
SCALE_TIER = "BASIC"

!gcloud ai-platform jobs submit training $JOB_NAME \
--region=$REGION \
--job-dir=$JOB_DIR \
--master-image-uri=$IMAGE_URI \
--scale-tier=$SCALE_TIER \
-- \
--training_dataset_path=$TRAINING_FILE_PATH \
--validation_dataset_path=$VALIDATION_FILE_PATH \
--alpha=$alpha \
--max_iter=$max_iter \
--nohptune

Job [JOB_20200204_031418] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe JOB_20200204_031418

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs JOB_20200204_031418
jobId: JOB_20200204_031418
state: QUEUED


In [24]:
!gcloud ai-platform jobs stream-logs $JOB_NAME

INFO	2020-02-04 03:14:19 +0000	service		Validating job requirements...
INFO	2020-02-04 03:14:20 +0000	service		Job creation request has been successfully validated.
INFO	2020-02-04 03:14:20 +0000	service		Job JOB_20200204_031418 is queued.
INFO	2020-02-04 03:14:20 +0000	service		Waiting for job to be provisioned.
INFO	2020-02-04 03:14:23 +0000	service		Waiting for training program to start.
INFO	2020-02-04 03:23:24 +0000	master-replica-0		Copying file://model.pkl [Content-Type=application/octet-stream]...
INFO	2020-02-04 03:23:25 +0000	master-replica-0		/ [0 files][    0.0 B/  6.1 KiB]                                                
INFO	2020-02-04 03:23:25 +0000	master-replica-0		/ [1 files][  6.1 KiB/  6.1 KiB]                                                
INFO	2020-02-04 03:23:25 +0000	master-replica-0		Operation completed over 1 objects/6.1 KiB.                                      
INFO	2020-02-04 03:23:25 +0000	master-replica-0		Starting training: alpha=9.024414062499999e-05, m

### Examine the training output

The training script saved the trained model as the 'model.pkl' in the `JOB_DIR` folder on GCS.

In [25]:
!gsutil ls $JOB_DIR

gs://mlops-env20-artifact-store/jobs/JOB_20200204_031418/model.pkl


## Deploy the model to AI Platform Prediction

### Create a model resource

In [28]:
model_name = "forest_cover_classifier"
labels = "task=classifier,domain=forestry"

!gcloud ai-platform models create  $model_name \
--regions=$REGION \
--labels=$labels

Created ml engine model [projects/mlops-workshop/models/forest_cover_classifier].


### Create a model version

In [29]:
model_version = 'v01'

!gcloud ai-platform versions create $model_version \
--model=$model_name \
--origin=$JOB_DIR \
--runtime-version=1.14 \
--framework=scikit-learn \
--python-version=3.5

Creating version (this might take a few minutes)......done.                    


### Serve predictions
#### Prepare the input file with JSON formated instances.

In [30]:
input_file = 'serving_instances.json'

with open(input_file, 'w') as f:
    for index, row in X_validation.head().iterrows():
        f.write(json.dumps(list(row.values)))
        f.write('\n')

In [31]:
!cat $input_file

[3092.0, 163.0, 13.0, 30.0, 5.0, 4886.0, 232.0, 243.0, 139.0, 1456.0, "Rawah", "C7745"]
[2998.0, 9.0, 6.0, 30.0, 0.0, 1819.0, 214.0, 228.0, 152.0, 1550.0, "Rawah", "C7745"]
[2070.0, 37.0, 30.0, 335.0, 221.0, 313.0, 203.0, 160.0, 74.0, 871.0, "Cache", "C2706"]
[2325.0, 28.0, 12.0, 0.0, 0.0, 1260.0, 215.0, 213.0, 133.0, 162.0, "Cache", "C2717"]
[3318.0, 96.0, 12.0, 95.0, -5.0, 1224.0, 239.0, 222.0, 111.0, 1411.0, "Neota", "C8771"]


#### Invoke the model

In [32]:
!gcloud ai-platform predict \
--model $model_name \
--version $model_version \
--json-instances $input_file

[2, 2, 3, 3, 1]
